In [6]:
import pandas as pd
import torch

# Load the MovieLens ratings data (assumed to have columns: userId, movieId, rating, timestamp)
ratings = pd.read_csv('ratings.csv')  # Path to the MovieLens dataset file
# (The dataset can be downloaded from the MovieLens website if not already available)

# Create contiguous indices for users and movies
user_ids = ratings['userId'].unique()
movie_ids = ratings['movieId'].unique()
num_users = len(user_ids)
num_items = len(movie_ids)
user_id_to_idx = {uid: idx for idx, uid in enumerate(user_ids)}
movie_id_to_idx = {mid: idx for idx, mid in enumerate(movie_ids)}
ratings['user_idx'] = ratings['userId'].map(user_id_to_idx)
ratings['movie_idx'] = ratings['movieId'].map(movie_id_to_idx)

# Split into train and test sets (leave-one-out by latest timestamp per user)
ratings = ratings.sort_values(['user_idx', 'timestamp'])
test_indices = ratings.groupby('user_idx').tail(1).index  # last interaction for each user
train_indices = ratings.index.difference(test_indices)
train_df = ratings.loc[train_indices].reset_index(drop=True)
test_df = ratings.loc[test_indices].reset_index(drop=True)

# Construct edge index for user-item graph using training interactions
# We use a homogeneous graph representation:
# user nodes 0..num_users-1 and item nodes num_users..num_users+num_items-1.
user_index_tensor = torch.tensor(train_df['user_idx'].values, dtype=torch.long)
item_index_tensor = torch.tensor(train_df['movie_idx'].values, dtype=torch.long)
item_index_tensor += num_users  # offset item indices
edge_index = torch.stack([user_index_tensor, item_index_tensor], dim=0)  # shape [2, num_train_edges]

# Prepare a dictionary of test interactions for evaluation (per user index)
test_interactions = test_df.groupby('user_idx')['movie_idx'].apply(list).to_dict()

# (Optionally) also prepare train interactions dict for convenience (e.g. for filtering known items)
train_interactions = train_df.groupby('user_idx')['movie_idx'].apply(list).to_dict()

In [13]:
import torch.nn as nn
from torch_geometric.nn import LGConv

class GNNRecommender(nn.Module):
    def __init__(self, num_users, num_items, embedding_dim=64, num_layers=3):
        super(GNNRecommender, self).__init__()
        self.num_users = num_users
        self.num_items = num_items
        self.total_nodes = num_users + num_items
        # Initial trainable embeddings for all users and items:
        self.embedding = nn.Embedding(self.total_nodes, embedding_dim)
        # Define graph convolution layers (LightGCN propagation layers):
        self.convs = nn.ModuleList([LGConv() for _ in range(num_layers)])

    def forward(self, edge_index):
        # Begin with the initial embeddings for all nodes
        x = self.embedding.weight  # shape [total_nodes, embedding_dim]
        all_layer_embeddings = [x]  # list to collect embeddings at each layer
        # GNN propagation through each layer
        for conv in self.convs:
            # LGConv performs normalized neighbor aggregation
            x = conv(x, edge_index)
            all_layer_embeddings.append(x)
        # Combine embeddings from all layers (including initial).
        # LightGCN uses a weighted sum; here we use an equal-weight average for simplicity.
        all_embeddings = torch.stack(all_layer_embeddings, dim=0)  # shape [num_layers+1, total_nodes, emb_dim]
        final_embeddings = all_embeddings.mean(dim=0)              # shape [total_nodes, emb_dim]
        # Split the combined embeddings back into user and item embeddings
        user_embeds = final_embeddings[:self.num_users]            # [num_users, emb_dim]
        item_embeds = final_embeddings[self.num_users:]            # [num_items, emb_dim]
        return user_embeds, item_embeds


In [28]:
import random
import torch
import torch.nn as nn
import torch.optim as optim

def train_model(model, edge_index, train_interactions, num_users, num_items, epochs=10, lr=0.01, batch_size=1024):
    print("[INFO] Starting optimized training...")

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    edge_index = edge_index.to(device)

    optimizer = optim.Adam(model.parameters(), lr=lr)
    model.train()

    # Precompute user->set of interacted items
    user_to_items = {u: set(items) for u, items in train_interactions.items()}

    # Precompute negatives once
    negatives_per_user = {
        u: list(set(range(num_items)) - set(items))
        for u, items in train_interactions.items()
    }

    for epoch in range(1, epochs + 1):
        print(f"\n[INFO] Epoch {epoch}/{epochs}")
        total_loss = 0.0
        triplets = []

        # === Build training triplets (u, i, j) ===
        for u, pos_items in train_interactions.items():
            neg_items = negatives_per_user.get(u, [])
            if not neg_items:
                continue
            for pos_item in pos_items:
                neg_item = random.choice(neg_items)
                triplets.append((u, pos_item, neg_item))

        print(f"[DEBUG] Sampled {len(triplets)} training triplets")

        # === Batch training ===
        for i in range(0, len(triplets), batch_size):
            batch = triplets[i:i+batch_size]
            u_batch = torch.tensor([u for u, _, _ in batch], dtype=torch.long, device=device)
            i_batch = torch.tensor([i for _, i, _ in batch], dtype=torch.long, device=device)
            j_batch = torch.tensor([j for _, _, j in batch], dtype=torch.long, device=device)

            # ❗ MOVE FORWARD PASS HERE
            user_emb, item_emb = model(edge_index)
            user_emb = user_emb.to(device)
            item_emb = item_emb.to(device)

            u_vec = user_emb[u_batch]
            i_vec = item_emb[i_batch]
            j_vec = item_emb[j_batch]

            pos_score = torch.sum(u_vec * i_vec, dim=1)
            neg_score = torch.sum(u_vec * j_vec, dim=1)

            loss = -torch.mean(torch.log(torch.sigmoid(pos_score - neg_score)))
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        avg_loss = total_loss / (len(triplets) // batch_size + 1)
        print(f"[INFO] Epoch {epoch} completed | Avg BPR Loss: {avg_loss:.4f}")


In [30]:
def recommend_topN(user_idx, model, edge_index, train_interactions, N=10):
    model.eval()
    device = next(model.parameters()).device  # Get model's device (CPU or CUDA)

    user_idx_tensor = torch.tensor([user_idx], dtype=torch.long).to(device)
    edge_index = edge_index.to(device)

    # Compute embeddings on the right device
    user_emb, item_emb = model(edge_index)

    scores = torch.matmul(user_emb[user_idx_tensor], item_emb.T).squeeze(0)

    scores = scores.cpu().detach().numpy()  # convert to numpy for sorting
    seen_items = set(train_interactions.get(user_idx, []))
    for seen in seen_items:
        scores[seen] = -1e9

    topN_indices = scores.argsort()[-N:][::-1]
    recommended_items = [int(idx) for idx in topN_indices]
    return recommended_items


In [32]:
import numpy as np

def evaluate_model(model, edge_index, test_interactions, train_interactions, K=10):
    model.eval()
    device = next(model.parameters()).device  # Detect model's device
    edge_index = edge_index.to(device)

    # Move model and data to the correct device
    user_emb, item_emb = model(edge_index)
    user_emb = user_emb.to(device)
    item_emb = item_emb.to(device)

    recalls = []
    ndcgs = []

    num_users = model.num_users
    for u in range(num_users):
        true_items = test_interactions.get(u, [])
        if not true_items:
            continue

        scores = torch.matmul(user_emb[u], item_emb.T)
        scores = scores.detach().cpu().numpy()  # convert to numpy for ranking

        # Filter out training items
        for train_item in train_interactions.get(u, []):
            scores[train_item] = -1e9

        topk_idx = np.argpartition(scores, -K)[-K:]
        topk_idx = topk_idx[np.argsort(scores[topk_idx])][::-1]

        # Evaluate Recall@K and NDCG@K
        true_set = set(true_items)
        hits = 0
        dcg = 0.0
        idcg = 0.0

        for rank, idx in enumerate(topk_idx):
            if idx in true_set:
                hits += 1
                dcg += 1.0 / np.log2(rank + 2)
        for rank in range(min(len(true_set), K)):
            idcg += 1.0 / np.log2(rank + 2)

        recalls.append(hits / len(true_set))
        ndcgs.append(dcg / idcg if idcg > 0 else 0.0)

    avg_recall = np.mean(recalls)
    avg_ndcg = np.mean(ndcgs)
    print(f"Recall@{K}: {avg_recall:.4f}, NDCG@{K}: {avg_ndcg:.4f}")
    return avg_recall, avg_ndcg


In [35]:
# Instantiate the model
model = GNNRecommender(num_users, num_items, embedding_dim=128, num_layers=3)

# Train the model on the training graph
train_model(model, edge_index, train_interactions, num_users, num_items, epochs=30, lr=0.005, batch_size=2048)

# Generate Top-5 recommendations for user with index 0
rec_movies = recommend_topN(user_idx=0, model=model, edge_index=edge_index, train_interactions=train_interactions, N=5)
print(f"Top-5 recommended movies for user 0: {rec_movies}")

# Evaluate the model performance with Recall@10 and NDCG@10
evaluate_model(model, edge_index, test_interactions, train_interactions, K=10)


[INFO] Starting optimized training...

[INFO] Epoch 1/30
[DEBUG] Sampled 100226 training triplets
[INFO] Epoch 1 completed | Avg BPR Loss: 0.8072

[INFO] Epoch 2/30
[DEBUG] Sampled 100226 training triplets
[INFO] Epoch 2 completed | Avg BPR Loss: 0.7751

[INFO] Epoch 3/30
[DEBUG] Sampled 100226 training triplets
[INFO] Epoch 3 completed | Avg BPR Loss: 0.7444

[INFO] Epoch 4/30
[DEBUG] Sampled 100226 training triplets
[INFO] Epoch 4 completed | Avg BPR Loss: 0.7168

[INFO] Epoch 5/30
[DEBUG] Sampled 100226 training triplets
[INFO] Epoch 5 completed | Avg BPR Loss: 0.6876

[INFO] Epoch 6/30
[DEBUG] Sampled 100226 training triplets
[INFO] Epoch 6 completed | Avg BPR Loss: 0.6577

[INFO] Epoch 7/30
[DEBUG] Sampled 100226 training triplets
[INFO] Epoch 7 completed | Avg BPR Loss: 0.6249

[INFO] Epoch 8/30
[DEBUG] Sampled 100226 training triplets
[INFO] Epoch 8 completed | Avg BPR Loss: 0.5901

[INFO] Epoch 9/30
[DEBUG] Sampled 100226 training triplets
[INFO] Epoch 9 completed | Avg BPR Los

(np.float64(0.02622950819672131), np.float64(0.011120877969428398))

In [34]:
# Save model state_dict
torch.save(model.state_dict(), "gnn_model.pt")
print("[INFO] Model saved to gnn_model.pt")


[INFO] Model saved to gnn_model.pt
